In [15]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
#generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
  in_text, result = seed_text, seed_text
  for i in range(n_words):
    #encode the text as interger
    encoded = tokenizer.text_to_sequences([in_text])[0]
    encoded = array(encoded)
    #predict a word in the vacabulary
    yhat  = model.predict_classes(encoded, verbose=0)
    #map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    #append to input
    intext, result = out_word, result+ ' '+out_word
  return result

In [3]:
data = """Bài thơ đầu anh viết tặng cho em
Là bài thơ anh kể về đôi dép
Khi nỗi nhớ ở trong lòng da diết
Những vật tầm thường cũng viết thành thơ

Hai chiếc dép kia gặp gỡ tự bao giờ
Có yêu nhau đâu mà chẳng rời nửa bước
Cùng gánh vác những nẻo đường xuôi ngược
Lên thảm nhung, xuống cát bụi, cùng nhau

Cùng bước, cùng mòn, không kẻ thấp người cao
Cùng chia sẻ sức người đời chà đạp
Dẫu vinh, nhục không đi cùng người khác
Số phận chiếc này phụ thuộc ở chiếc kia.

Nếu ngày nào một chiếc dép mất đi
Mọi thay thế đều trở nên khập khiễng
Giống nhau lắm nhưng người đi sẽ biết
Hai chiếc này chẳng phải một đôi đâu

Cũng như mình trong những lúc vắng nhau
Bước hụt hẫng cứ nghiêng về một phía
Dẫu bên cạnh đã có người thay thế
Mà trong lòng nỗi nhớ cứ chênh vênh

Đôi dép vô tri khăng khít bước song hành
Chẳng thề nguyện mà không hề giả dối
Chẳng hứa hẹn mà không hề phản bội
Lối đi nào cũng có mặt cả đôi.

Không thể thiếu nhau trên những bước đường đời
Dẫu mỗi chiếc ở một bên phải - trái
Như tôi yêu em ở những điều ngược lại
Gắn bó đời nhau vì một lối đi chung.

Hai mảnh đời thầm lặng bước song song
Sẽ dừng lại khi chỉ còn một chiếc
Chỉ còn một là không còn gì hết
Nếu không tìm được chiếc thứ hai kia."""

In [4]:
#integer encode text
tok = Tokenizer()
tok.fit_on_texts([data])
encoded = tok.texts_to_sequences([data])[0]
print(encoded)
#determine the vocabulary size
vocab_size = len(tok.word_index) + 1
print('vocabulary size: ', vocab_size)
#create word -> word sequences
sequences = list()
for i in range(1,len(encoded)):
  sequence = encoded[i-1:i+1]
  sequences.append(sequence)
print('total sequence: ', len(sequences))

[25, 17, 53, 26, 27, 54, 55, 28, 29, 25, 17, 26, 56, 30, 10, 11, 31, 32, 33, 12, 18, 34, 57, 58, 7, 59, 60, 61, 19, 27, 62, 17, 13, 1, 11, 20, 63, 64, 65, 66, 67, 21, 35, 4, 36, 14, 15, 68, 69, 5, 6, 70, 71, 7, 72, 37, 73, 38, 74, 75, 76, 77, 78, 79, 6, 4, 6, 5, 6, 80, 2, 81, 82, 8, 83, 6, 84, 85, 86, 8, 16, 87, 88, 22, 89, 90, 2, 9, 6, 8, 91, 92, 93, 1, 39, 94, 95, 12, 1, 20, 40, 96, 41, 3, 1, 11, 97, 9, 98, 42, 43, 99, 100, 101, 102, 103, 104, 4, 105, 106, 8, 9, 44, 107, 13, 1, 39, 15, 45, 3, 10, 36, 19, 46, 108, 18, 7, 109, 110, 4, 5, 111, 112, 47, 113, 30, 3, 114, 22, 48, 115, 116, 21, 8, 42, 43, 14, 18, 34, 32, 33, 47, 117, 118, 10, 11, 119, 120, 121, 122, 5, 23, 123, 15, 124, 125, 14, 2, 49, 126, 127, 15, 128, 129, 14, 2, 49, 130, 131, 50, 9, 41, 19, 21, 132, 133, 10, 2, 134, 135, 4, 136, 7, 5, 37, 16, 22, 137, 1, 12, 3, 48, 45, 138, 46, 139, 35, 28, 12, 7, 140, 38, 51, 141, 142, 16, 4, 143, 3, 50, 9, 144, 13, 145, 16, 146, 147, 5, 23, 23, 44, 148, 51, 31, 52, 24, 3, 1, 52, 24, 3

In [9]:
#split into X and y elements
sequences = np.array(sequences)
X, y = sequences[:,0], sequences[:,1]
#one hot encode outputs 
y = to_categorical(y, num_classes=vocab_size)

In [10]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
X[:5]

array([25, 17, 53, 26, 27])

In [17]:
#define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             1540      
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 154)               7854      
Total params: 21,594
Trainable params: 21,594
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
#compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X,y, epochs=500, verbose=2)

Epoch 1/500
9/9 - 0s - loss: 5.0373 - accuracy: 0.0228
Epoch 2/500
9/9 - 0s - loss: 5.0338 - accuracy: 0.0380
Epoch 3/500
9/9 - 0s - loss: 5.0311 - accuracy: 0.0380
Epoch 4/500
9/9 - 0s - loss: 5.0282 - accuracy: 0.0380
Epoch 5/500
9/9 - 0s - loss: 5.0252 - accuracy: 0.0380
Epoch 6/500
9/9 - 0s - loss: 5.0218 - accuracy: 0.0418
Epoch 7/500
9/9 - 0s - loss: 5.0181 - accuracy: 0.0570
Epoch 8/500
9/9 - 0s - loss: 5.0137 - accuracy: 0.0532
Epoch 9/500
9/9 - 0s - loss: 5.0088 - accuracy: 0.0570
Epoch 10/500
9/9 - 0s - loss: 5.0027 - accuracy: 0.0570
Epoch 11/500
9/9 - 0s - loss: 4.9955 - accuracy: 0.0570
Epoch 12/500
9/9 - 0s - loss: 4.9872 - accuracy: 0.0570
Epoch 13/500
9/9 - 0s - loss: 4.9771 - accuracy: 0.0570
Epoch 14/500
9/9 - 0s - loss: 4.9643 - accuracy: 0.0494
Epoch 15/500
9/9 - 0s - loss: 4.9495 - accuracy: 0.0380
Epoch 16/500
9/9 - 0s - loss: 4.9324 - accuracy: 0.0304
Epoch 17/500
9/9 - 0s - loss: 4.9115 - accuracy: 0.0304
Epoch 18/500
9/9 - 0s - loss: 4.8870 - accuracy: 0.0304
E

In [33]:
#evaluate
tokenizer = Tokenizer()
print(generate_seq(model,tokenizer,1,5))

AttributeError: ignored